In [1]:
import numpy as np
from datascience import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import pandas as pd
import warnings
warnings.simplefilter('ignore', FutureWarning)

/usr/local/lib/python3.6/dist-packages/datascience/tables.py:17: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)
/usr/local/lib/python3.6/dist-packages/datascience/util.py:10: MatplotlibDeprecationWarning: The 'warn' parameter of use() is deprecated since Matplotlib 3.1 and will be removed in 3.3.  If any parameter follows 'warn', they should be pass as keyword, not positionally.
  matplotlib.use('agg', warn=False)


In [3]:
mental_health = pd.read_csv('mental_health.csv')

mental_health = mental_health[(mental_health['Question 3']=='United States') | (mental_health['Question 3']=='United States of America')]

mental_health["Question 89"]= mental_health["Question 89"].replace(["European American","Caucasian"], "White") 
mental_health["Question 89"]= mental_health["Question 89"].replace(["White Hispanic"], "Hispanic") 
mental_health["Question 89"]= mental_health["Question 89"].replace(["More than one of the above"], "Mixed Race") 
mental_health["Question 89"]= mental_health["Question 89"].replace(["Hispanic"], np.NAN) 
mental_health["Question 89"]= mental_health["Question 89"].replace(["American Indian or Alaska Native"], np.NAN) 
mental_health["Question 89"]= mental_health["Question 89"].replace(["Black or African American"],"Black") 

In [5]:
def two_samples_proportions_diagnosed(question,var1=0,var2=1):
    try:
        question_poss_answers = np.unique(mental_health[question][(~pd.isna(mental_health[question]))&(~pd.isna(mental_health['Question 34']))])
        
        diagnosed_ans1 = mental_health[question][(mental_health[question]==question_poss_answers[var1])&(mental_health['Question 34']=='Yes')]
        not_diagnosed_ans1 = mental_health[question][(mental_health[question]==question_poss_answers[var1])&(mental_health['Question 34']=='No')]
        
        diagnosed_ans2 = mental_health[question][(mental_health[question]==question_poss_answers[var2])&(mental_health['Question 34']=='Yes')]
        not_diagnosed_ans2 = mental_health[question][(mental_health[question]==question_poss_answers[var2])&(mental_health['Question 34']=='No')]
        
        
        ans1 = np.append(diagnosed_ans1,not_diagnosed_ans1)
        ans1_diagstat = ['Diagnosed']*len(diagnosed_ans1) + ['Not Diagnosed']*len(not_diagnosed_ans1)
        
        ans2 = np.append(diagnosed_ans2,not_diagnosed_ans2)
        ans2_diagstat = ['Diagnosed']*len(diagnosed_ans2) + ['Not Diagnosed']*len(not_diagnosed_ans2)
        
        
        total = np.append(ans1_diagstat,ans2_diagstat)
        
        sim_diff = make_array()
        
        for i in range(1000):
            shuffled_diag = np.random.permutation(total)
            #counting proportion of randomized diagnosed people for that who anwered with choice 1
            answer1_diagnosed = sum(shuffled_diag[0:len(ans1)]=='Diagnosed')/len(ans1)
                    
            #counting proportion of randomized diagnosed people for that who anwered with choice 2
            answer2_diagnosed = sum(shuffled_diag[len(ans1):len(ans1)+len(ans2)]=='Diagnosed')/len(ans2)
            
            sim_diff = np.append(sim_diff,(answer1_diagnosed-answer2_diagnosed))
            
        obs_diff = (len(diagnosed_ans1)/(len(diagnosed_ans1)+len(not_diagnosed_ans1)))-(len(diagnosed_ans2)/(len(diagnosed_ans2)+len(not_diagnosed_ans2)))
        
        print('For determination of significance of difference between proportions of diagnosed individuals between those who responded "',question_poss_answers[var1] ,'" and "',question_poss_answers[var2],'" for',question,', observed difference between proportion diagnosed of "',question_poss_answers[var1],'" individuals and "',question_poss_answers[var2],'" responding individuals (former minus later) is ',obs_diff)
        
        if obs_diff<np.mean(sim_diff):
            one_sided_p_value = sum(obs_diff>sim_diff)/1000
            two_sided_p_value = 2*one_sided_p_value
        else:
            one_sided_p_value = sum(obs_diff<sim_diff)/1000
            two_sided_p_value = 2*one_sided_p_value
        
        print('One sided p_value = ',one_sided_p_value,', two sided p_value = ',two_sided_p_value)
   
        return(one_sided_p_value,two_sided_p_value)
        
    except Exception as e:
        print(e)
        return

In [6]:
two_samples_proportions_diagnosed('Question 13')

For determination of significance of difference between proportions of diagnosed individuals between those who responded " No " and " Yes " for Question 13 , observed difference between proportion diagnosed of " No " individuals and " Yes " responding individuals (former minus later) is  -0.07395434753925323
One sided p_value =  0.033 , two sided p_value =  0.066


(0.033, 0.066)

In [7]:
two_samples_proportions_diagnosed('Question 5')

For determination of significance of difference between proportions of diagnosed individuals between those who responded " No " and " Yes " for Question 5 , observed difference between proportion diagnosed of " No " individuals and " Yes " responding individuals (former minus later) is  0.019626190196308957
One sided p_value =  0.258 , two sided p_value =  0.516


(0.258, 0.516)

In [8]:
mental_health["MaleFemale or Other"]= mental_health["Question 2"].replace(["Male","Female"], "Male/Female") 

In [9]:
two_samples_proportions_diagnosed('Question 2')

For determination of significance of difference between proportions of diagnosed individuals between those who responded " Female " and " Male " for Question 2 , observed difference between proportion diagnosed of " Female " individuals and " Male " responding individuals (former minus later) is  0.15961946681650752
One sided p_value =  0.0 , two sided p_value =  0.0


(0.0, 0.0)

In [10]:
two_samples_proportions_diagnosed('MaleFemale or Other')

For determination of significance of difference between proportions of diagnosed individuals between those who responded " Male/Female " and " Other " for MaleFemale or Other , observed difference between proportion diagnosed of " Male/Female " individuals and " Other " responding individuals (former minus later) is  -0.225604311008468
One sided p_value =  0.0 , two sided p_value =  0.0


(0.0, 0.0)